# TP — Modèles statistiques pour séries temporelles (ARIMA, SARIMA, Auto-ARIMA) avec `statsmodels`

Ce notebook est conçu pour être exécuté **en local** ou sur **Google Colab**.

🎯 Objectif : apprendre les bases des modèles statistiques de forecasting sur une **série standard**,
et comprendre l'impact des choix de modèles (ARIMA vs SARIMA) sur les performances.

Nous utiliserons le dataset classique **AirPassengers** (nombre de passagers aériens mensuels).
Il contient une tendance + une saisonnalité annuelle marquée, idéal pour débuter.

---

## Plan
1. Chargement et visualisation des données
2. Découpage train/test + métriques (MAE, RMSE, MAPE)
3. Décomposition (tendance / saisonnalité / résidu)
4. Stationnarité et transformations (log, différenciation)
5. Modèles : Baseline, ARIMA, SARIMA
6. **Auto-ARIMA / Auto-SARIMA** (sélection automatique des paramètres)
7. Comparaison + interprétation + exercices

> Conseils Colab : *Runtime → CPU (suffisant)*


In [ ]:
# Installation (Colab / local)
!pip -q install -U statsmodels pmdarima pandas numpy matplotlib


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

import pmdarima as pm

plt.rcParams["figure.figsize"] = (12, 4)


## 1) Charger un dataset standard : AirPassengers

Le dataset AirPassengers (mensuel) est souvent utilisé pour illustrer :
- une **tendance** (le trafic augmente sur le long terme)
- une **saisonnalité** (pics récurrents chaque année)

Nous allons : charger → mettre un index temporel → visualiser.


In [ ]:
# Dataset AirPassengers via statsmodels
data = sm.datasets.airline.load_pandas().data.copy()
data.head()


In [ ]:
# Préparer un DataFrame propre avec un index datetime
df = data.rename(columns={"passengers": "y"})
df["month"] = pd.to_datetime(df["month"])
df = df.set_index("month").asfreq("MS")  # fréquence mensuelle (month start)
df.head()


In [ ]:
# Visualiser la série
plt.plot(df.index, df["y"], label="Passagers")
plt.title("AirPassengers — série mensuelle")
plt.xlabel("Date")
plt.ylabel("Nombre de passagers")
plt.grid(True, alpha=0.2)
plt.show()


## 2) Décomposition : tendance / saisonnalité / résidu

Pour comprendre visuellement :
- tendance (long terme)
- saisonnalité (périodique)
- bruit (résidu)

Ici, une décomposition **multiplicative** est adaptée (la variance augmente avec le niveau).


In [ ]:
decomp = seasonal_decompose(df["y"], model="multiplicative", period=12)
decomp.plot()
plt.suptitle("Décomposition multiplicative (période = 12 mois)", y=1.02)
plt.show()


## 3) Découpage train/test + métriques

On garde les **derniers mois** comme test (simulation d'une vraie prévision).

Métriques :
- **MAE** : erreur absolue moyenne
- **RMSE** : pénalise plus les grosses erreurs
- **MAPE** : erreur en % (attention si valeurs proches de 0)

👉 Baseline :
- **naïve** : la prévision = dernière valeur observée (référence minimale).


In [ ]:
def mae(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mape(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    eps = 1e-9
    return np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100


In [ ]:
# Split
TEST_SIZE = 24  # 24 derniers mois
train = df.iloc[:-TEST_SIZE].copy()
test  = df.iloc[-TEST_SIZE:].copy()

print("Train:", train.index.min(), "→", train.index.max(), "n=", len(train))
print("Test :", test.index.min(), "→", test.index.max(), "n=", len(test))


In [ ]:
# Baseline naive: y_hat(t) = dernière valeur observée
naive_pred = np.repeat(train["y"].iloc[-1], len(test))

print("Baseline naive")
print("MAE :", mae(test["y"], naive_pred))
print("RMSE:", rmse(test["y"], naive_pred))
print("MAPE:", mape(test["y"], naive_pred))


In [ ]:
# Plot baseline
plt.plot(train.index, train["y"], label="train")
plt.plot(test.index, test["y"], label="test")
plt.plot(test.index, naive_pred, label="baseline naive")
plt.title("Baseline naive (dernière valeur)")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()


## 4) Stationnarité (intuition) + test ADF

ARIMA suppose que la série est (au moins approximativement) stationnaire après transformations.

Test ADF (Augmented Dickey-Fuller) :
- H0 : *la série a une racine unitaire* (non-stationnaire)
- si p-value < 0.05 → on rejette H0 → plutôt stationnaire

⚠️ C'est un indicateur, pas une vérité absolue.


In [ ]:
def adf_test(series, name=""):
    result = adfuller(series.dropna(), autolag="AIC")
    print(f"ADF test {name}")
    print("  ADF statistic:", result[0])
    print("  p-value      :", result[1])
    print("  lags used    :", result[2])
    print("  nobs         :", result[3])
    print()

adf_test(train["y"], "sur y (train)")


### Transformations utiles : log + différenciation

- **log(y)** stabilise la variance (utile quand l'amplitude augmente avec le niveau)
- **diff(log(y))** aide à enlever la tendance (d=1)

On observe ensuite si la stationnarité s'améliore.


In [ ]:
# Log-transform
train_log = np.log(train["y"])
test_log  = np.log(test["y"])

# Différence (d=1)
train_log_diff = train_log.diff()

adf_test(train_log, "log(y)")
adf_test(train_log_diff, "diff(log(y))")


In [ ]:
plt.figure(figsize=(12,3))
plt.plot(train_log, label="log(y)")
plt.title("Train — log(y)")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()

plt.figure(figsize=(12,3))
plt.plot(train_log_diff, label="diff(log(y))")
plt.title("Train — diff(log(y))")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()


## 5) ARIMA (non saisonnier)

Notation : ARIMA(p, d, q)

Pour débuter, on choisit un modèle simple : **ARIMA(1,1,1)** sur **log(y)**, puis on revient à l'échelle originale via `exp`.


In [ ]:
order_arima = (1, 1, 1)

model_arima = ARIMA(train_log, order=order_arima)
res_arima = model_arima.fit()
print(res_arima.summary())


In [ ]:
# Forecast ARIMA
fc_arima_log = res_arima.get_forecast(steps=len(test)).predicted_mean
fc_arima = np.exp(fc_arima_log)

print("ARIMA", order_arima)
print("MAE :", mae(test["y"], fc_arima))
print("RMSE:", rmse(test["y"], fc_arima))
print("MAPE:", mape(test["y"], fc_arima))


In [ ]:
plt.plot(train.index, train["y"], label="train")
plt.plot(test.index, test["y"], label="test")
plt.plot(test.index, fc_arima, label=f"ARIMA{order_arima}")
plt.title("Prévision ARIMA (sur log(y))")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()


## 6) SARIMA (saisonnier) via SARIMAX

SARIMA(p,d,q)×(P,D,Q,s) ajoute une composante saisonnière.

Configuration classique de départ pour AirPassengers :
- order = (1, 1, 1)
- seasonal_order = (1, 1, 1, 12)

On entraîne sur **log(y)**.


In [ ]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)

sarima = SARIMAX(
    train_log,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)
res_sarima = sarima.fit(disp=False)
print(res_sarima.summary())


In [ ]:
# Forecast SARIMA
fc_sarima_log = res_sarima.get_forecast(steps=len(test)).predicted_mean
fc_sarima = np.exp(fc_sarima_log)

print("SARIMA", order, "x", seasonal_order)
print("MAE :", mae(test["y"], fc_sarima))
print("RMSE:", rmse(test["y"], fc_sarima))
print("MAPE:", mape(test["y"], fc_sarima))


In [ ]:
plt.plot(train.index, train["y"], label="train")
plt.plot(test.index, test["y"], label="test")
plt.plot(test.index, fc_arima, label=f"ARIMA{order_arima}")
plt.plot(test.index, fc_sarima, label=f"SARIMA{order}x{seasonal_order}")
plt.title("Comparaison ARIMA vs SARIMA (sur log(y))")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()


## 7) Auto-ARIMA / Auto-SARIMA (sélection automatique des paramètres)

Au lieu de choisir (p,d,q) à la main, **Auto-ARIMA** explore plusieurs configurations et sélectionne un modèle selon un critère (souvent **AIC**).

✅ Avantages :
- bon point de départ pour débuter
- évite de tester trop de modèles manuellement

⚠️ À retenir :
- Auto-ARIMA ne remplace pas l'analyse : il faut vérifier les résidus et la cohérence du modèle.


In [ ]:
# Auto-ARIMA (non saisonnier) sur log(y)
auto_arima_model = pm.auto_arima(
    train_log,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    error_action="ignore",
    trace=True
)

print(auto_arima_model.summary())
print("Best ARIMA order:", auto_arima_model.order)


In [ ]:
# Forecast Auto-ARIMA
fc_auto_arima_log = auto_arima_model.predict(n_periods=len(test))
fc_auto_arima = np.exp(fc_auto_arima_log)

print("Auto-ARIMA", auto_arima_model.order)
print("MAE :", mae(test["y"], fc_auto_arima))
print("RMSE:", rmse(test["y"], fc_auto_arima))
print("MAPE:", mape(test["y"], fc_auto_arima))


In [ ]:
# Auto-SARIMA (saisonnier) sur log(y) avec m=12
auto_sarima_model = pm.auto_arima(
    train_log,
    seasonal=True,
    m=12,
    stepwise=True,
    suppress_warnings=True,
    error_action="ignore",
    trace=True
)

print(auto_sarima_model.summary())
print("Best SARIMA order:", auto_sarima_model.order)
print("Best seasonal order:", auto_sarima_model.seasonal_order)


In [ ]:
# Forecast Auto-SARIMA
fc_auto_sarima_log = auto_sarima_model.predict(n_periods=len(test))
fc_auto_sarima = np.exp(fc_auto_sarima_log)

print("Auto-SARIMA", auto_sarima_model.order, "x", auto_sarima_model.seasonal_order)
print("MAE :", mae(test["y"], fc_auto_sarima))
print("RMSE:", rmse(test["y"], fc_auto_sarima))
print("MAPE:", mape(test["y"], fc_auto_sarima))


In [ ]:
# Plot comparaison (tous)
plt.figure(figsize=(12,4))
plt.plot(train.index, train["y"], label="train")
plt.plot(test.index, test["y"], label="test")

plt.plot(test.index, naive_pred, label="Naive")
plt.plot(test.index, fc_arima, label=f"ARIMA{order_arima}")
plt.plot(test.index, fc_sarima, label=f"SARIMA{order}x{seasonal_order}")
plt.plot(test.index, fc_auto_arima, label=f"Auto-ARIMA{auto_arima_model.order}")
plt.plot(test.index, fc_auto_sarima, label=f"Auto-SARIMA{auto_sarima_model.order}x{auto_sarima_model.seasonal_order}")

plt.title("Comparaison : Baseline vs ARIMA/SARIMA vs Auto-ARIMA")
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()


## 8) Comparaison synthétique (tableau)

On compare baseline, ARIMA, SARIMA, Auto-ARIMA et Auto-SARIMA.


In [ ]:
results = pd.DataFrame({
    "Model": [
        "Naive",
        f"ARIMA{order_arima}",
        f"SARIMA{order}x{seasonal_order}",
        f"Auto-ARIMA{auto_arima_model.order}",
        f"Auto-SARIMA{auto_sarima_model.order}x{auto_sarima_model.seasonal_order}",
    ],
    "MAE": [
        mae(test["y"], naive_pred),
        mae(test["y"], fc_arima),
        mae(test["y"], fc_sarima),
        mae(test["y"], fc_auto_arima),
        mae(test["y"], fc_auto_sarima),
    ],
    "RMSE": [
        rmse(test["y"], naive_pred),
        rmse(test["y"], fc_arima),
        rmse(test["y"], fc_sarima),
        rmse(test["y"], fc_auto_arima),
        rmse(test["y"], fc_auto_sarima),
    ],
    "MAPE(%)": [
        mape(test["y"], naive_pred),
        mape(test["y"], fc_arima),
        mape(test["y"], fc_sarima),
        mape(test["y"], fc_auto_arima),
        mape(test["y"], fc_auto_sarima),
    ],
})
results


## 9) Diagnostics résiduels (idée)

Un bon modèle laisse des résidus proches du **bruit blanc** :
- pas de structure temporelle forte
- autocorrélations faibles

On regarde les résidus de SARIMA (manuel) comme exemple.


In [ ]:
resid = res_sarima.resid.dropna()

plt.figure(figsize=(12,3))
plt.plot(resid)
plt.title("Résidus SARIMA (sur log-scale)")
plt.grid(True, alpha=0.2)
plt.show()

from statsmodels.graphics.tsaplots import plot_acf
plot_acf(resid, lags=36)
plt.title("ACF des résidus SARIMA")
plt.show()


---
# Exercices (pour aller plus loin)

1) **Choix des paramètres** : essayez ARIMA(2,1,2) et SARIMA(2,1,2)×(1,1,1,12).
2) **Auto-ARIMA** : comparez `stepwise=True` vs `stepwise=False` (plus lent mais plus exhaustif).
3) **Impact du test size** : changez `TEST_SIZE` (12, 36).
4) **Prévision plus longue** : prévoyez 36 mois et visualisez.
5) **Interprétation** : quel modèle choisiriez-vous et pourquoi (métriques + simplicité) ?


## Fin

Vous avez maintenant une base solide sur :
- visualisation & décomposition,
- split train/test,
- ARIMA vs SARIMA,
- Auto-ARIMA / Auto-SARIMA,
- évaluation et diagnostics.

Prochaine étape naturelle :
- ETS (Holt-Winters), Prophet, puis modèles deep learning.
